# Computational Finance Assignment1

In [ ]:
import math
import sympy as sp
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt


## Part I Theory: derivatives and no-arbitrage

### 1.Prove:

`A = C(1 + r/n)^(nt) = C * e^(rt)`

The standard compound interest formula is: 

`A = C(1 + r/n)^(nt)`, where:

- `A` is the future value,
- `C` is the principal (initial investment),
- `r` is the annual interest rate,
- `n` is the number of compound interest per year,
- `t` is time in years.

As we let `n` go to infinity, we can use the property of the base `e` of the natural logarithm, `(1 + 1/n)^n` tends to `e` as `n` goes to infinity. We set `n = 1,10,100,1000,10000` to compare these two results


In [ ]:
C = 1 
r = 0.1
t = 1

A_true = C * math.exp(r * t) 
ns = [1,10,100,1000,10000]
for n in ns:
    A_n= C * pow(1 + r/n, n * t)
    absolute_error = abs(A_true - A_n)/A_true
    print(f"Continuous compound interest : {A_true} ", f"Standard compound interest (with n = {n}): {A_n}")
    print(f"Absolute Error:{absolute_error}")
    print("\n")


### 2.Calculate the fair-value of a coupon bond.

In [ ]:


# Calculate the fair value of a bond
# Given parameters
face_value = 50000  # Face value of the bond
coupon = 300  # Coupon payment per quarter
maturity = 2  # Maturity period in years
risk_free_rate = 0.015  # Risk-free interest rate
quarters = maturity * 4  # Total number of quarters for payments

# Calculate the present value of each quarter
present_value_coupons = sum(coupon * sp.exp(-risk_free_rate * t/4) for t in range(1, quarters + 1))

# Present value of the final principal payment
present_value_face_value = face_value * sp.exp(-risk_free_rate * maturity)

# The fair value of the bond is the sum of present values of all cash flows
fair_value_bond = present_value_coupons + present_value_face_value

print(f"Fair-value of this coupon bond : {fair_value_bond.evalf()}")  # Evaluate the numerical result of the expression




### 3.Calculate forward price of a contract at time zero

In financial markets, the fair forward price of a contract at time zero is often calculated using the no-arbitrage principle. This principle dictates that the price of derivative securities must be set in a way that does not allow for arbitrage opportunities through risk-free trades with guaranteed profits. The fair forward price, denoted by `F_0`, is expressed as `S_0 * e^(rT)`, which indicates that the forward price is equivalent to the current price of the underlying asset, `S_0`, compounded at the risk-free interest rate r over the time to maturity T.

Thus, if the actual forward price in the market deviates from `S_0 * e^(rT)`, arbitrageurs can exploit this difference to secure risk-free profits. 

If the market forward price is higher than `S_0 * e^(rT)`, it is profitable to sell the forward contract and invest the current underlying asset price `S_0` at the risk-free rate r. 

Conversely, if the market forward price is lower than `S_0 * e^(rT)`, one can achieve a risk-free profit by entering into a forward contract to buy the asset and simultaneously borrowing the amount `S_0` to purchase the underlying asset. This process will continue until the market forward price adjusts to match the no-arbitrage price of `S_0 * e^(rT)`.

### 4. Draw the pay-oﬀ diagrams for two portfolios

1. A call option and an investment of `Ke^{-rT}` in the money-market; 
2. A put option and one share of the stock `S`.


In [ ]:

# Define the parameters
K = 50  # Strike price
r = 0.05  # Risk-free rate
T = 1  # Time to maturity
S0 = 50  # Current stock price

# Define the range of stock prices at maturity
ST = np.linspace(0, 100, 500)

# Calculate the payoff for a call option at maturity
call_payoff = np.maximum(ST - K, 0)
# Calculate the present value of the investment in the money market
money_market_investment = K * np.exp(-r * T)
# Total payoff for the first portfolio (call option + money market investment)
portfolio1_payoff = call_payoff + money_market_investment

# Calculate the payoff for a put option at maturity
put_payoff = np.maximum(K - ST, 0)
# Total payoff for the second portfolio (put option + stock)
portfolio2_payoff = put_payoff + ST

# Plot the pay-off diagrams for both portfolios
plt.figure(figsize=(14, 7))

# Portfolio 1: Call option and money market investment
plt.subplot(1, 2, 1)
plt.plot(ST, portfolio1_payoff, label='Call Option + Money Market Investment')
plt.title('Portfolio 1 Payoff at Maturity')
plt.xlabel('Stock Price at Maturity (ST)')
plt.ylabel('Profit')
plt.legend()
plt.grid(True)

# Portfolio 2: Put option and stock
plt.subplot(1, 2, 2)
plt.plot(ST, portfolio2_payoff, label='Put Option + Stock')
plt.title('Portfolio 2 Payoff at Maturity')
plt.xlabel('Stock Price at Maturity (ST)')
plt.ylabel('Profit')
plt.legend()
plt.grid(True)

# Show the plots
plt.tight_layout()
plt.show()


Portfolio 1 Payoff at Maturity (Call Option + Money Market Investment): This graph shows that if the stock price at maturity (ST) is below the strike price (K), the payoff is constant because the call option is not exercised and the investor only gets the money market investment value. As the stock price exceeds the strike price, the payoff increases linearly with ST since the call option is exercised.

Portfolio 2 Payoff at Maturity (Put Option + Stock): This graph illustrates that if the stock price at maturity is above the strike price, the payoff is linear and increases with ST because the put option is not exercised, and the investor owns the stock. When the stock price is below the strike price, the payoff is constant because the put option is exercised, ensuring the investor sells the stock at the strike price K.

The linear payoff beyond the strike price reflects the characteristics of owning the underlying asset (stock) with the protection of an option. The flat part of the payoff represents the protective feature of the option where the loss is limited to the premium paid for the option. ​

### 5. Prove put-call parity：`C_t + Ke^(-r(T-t)) = P_t + S_0`
#### Portfolio A:
- Consists of one European call option with the value `C_t` and an investment of `Ke^(-r(T-t))` in the risk-free asset. The payoff at maturity is the greater of `S_T - K` or 0, plus `K`, which simplifies to `max(S_T, K)`.

#### Portfolio B:
- Consists of one European put option with the value `P_t` and one share of the stock `S_0`. The payoff at maturity is the greater of `K - S_T` or 0, plus `S_T`, which simplifies to `max(S_T, K)`.

Given that the payoff of both portfolios is the same at maturity, and no arbitrage opportunities exist, their value should be equal at the present time. This leads to the equation:

`C_t + Ke^(-r(T-t)) = P_t + S_0`


## Part II: Binomial tree: option valuation

#### 1. Build the binomial rree and test

In [ ]:
def buildTree(S, vol, T, N):
    '''
    Build a binomial tree for option pricing.
    Paramers:
        S: Initial stock price
        vol: Volatility
        T: Time to expiration
        N: Number of steps
    Output:
        A 2D array representing the binomial tree
    '''
    dt = T / N  # Time step
    u = np.exp(vol * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    matrix = np.zeros((N + 1, N + 1))  # Create a 2D array for tree to store the stock prices
    
    for i in range(N + 1): 
        for j in range(i + 1):  
            matrix[i, j] = S * (u ** j) * (d ** (i - j))
            
    return matrix

def valueOptionMatrix(tree, T, r, K, vol):
    '''
    Calculate the option value using a binomial tree.
    Parameters:
        tree: Binomial tree
        T: Time to expiration
        r: Risk-free interest rate
        K: Strike price
        vol: Volatility
    Output:
        Option value at the root of the tree
    '''
    N = tree.shape[0] - 1  # Number of steps
    dt = T / N 
    u = np.exp(vol * np.sqrt(dt)) 
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    for c in range(N + 1):
        tree[N, c] = max(0, tree[N, c] - K) # Option value at expiration
    
    for i in range(N - 1, -1, -1): 
        for j in range(i + 1):
            # Calculate the expected value of the option
            tree[i, j] = np.exp(-r * dt) * (p * tree[i + 1, j + 1] + (1 - p) * tree[i + 1, j])
            
    return tree[0, 0]

In [ ]:
sigma = 0.2  # Volatility
S = 100  # Current stock price
T = 1.0  # Time to maturity
N = 50  # Number of steps
K = 99  # Strike price
r = 0.06  # Risk-free interest rate

tree = buildTree(S, sigma, T, N)
option_price = valueOptionMatrix(tree, T, r, K, sigma)
option_price

#### 2.  Compare with the Black-Scholes fomula on value of option

In [ ]:
def black_scholes_call(S, K, T, r, sigma):
    """
    Computes the Black-Scholes formula for a European call option.
    
    Parameters:
        S : float
            Current stock price
        K : float
            Strike price
        T : float
            Time to expiration
        r : float
            Risk-free interest rate
        sigma : float
            Volatility of the underlying stock
    
    Returns:
        call_price : float
            Price of the European call option
    """
    # Calculate d1 and d2 parameters
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Calculate call price using cumulative distribution functions
    call_price = (S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)) 
    
    return call_price

def compare_volatilities(S, K, T, r, vol_range, N):
    '''
    Compare option prices for different volatilities.
    Parameters:
        S : float
            Current stock price
        K : float
            Strike price
        T : float
            Time to expiration
        r : float
            Risk-free interest rate
        vol_range : array
            Array of volatilities
    Returns:
        bs_prices : array
            Array of option prices for different volatilities under Black-Scholes model and binomial tree model
    '''
    bs_prices = []
    bt_prices = []
    for sigma in vol_range:
        bs_price = black_scholes_call(S, K, T, r, sigma)
        bs_prices.append(bs_price)
        tree = buildTree(S, sigma, T, N)
        option_price = valueOptionMatrix(tree, T, r, K, sigma)
        bt_prices.append(option_price)
    return bs_prices, bt_prices


In [ ]:
vol_range = np.linspace(0.1, 0.5, 100)

bs_prices, binomial_prices = compare_volatilities(S, K, T, r, vol_range, N)

# Plotting both Black-Scholes and Binomial tree prices for comparison
plt.figure(figsize=(10, 6))
plt.plot(vol_range, bs_prices, label='Black-Scholes Prices')
plt.plot(vol_range, binomial_prices, label='Binomial Tree Prices', linestyle='--')
plt.title('European Call Option Price for Different Volatilities')
plt.xlabel('Volatility')
plt.ylabel('Call Option Price')
plt.legend()
plt.grid(True)
plt.show()

## Part III Black-Scholes: hedging simulations

### 1.Prove: ∆t = $N\left(d_{1}\right)$

From the appendix we can prove that V(t) can be expressed as follows

$\begin{aligned}
V(t)& =\quad e^{-r\tau}\mathbb{E}\left[(S_{T}-K)^{+}\right]  \\
&=\quad e^{-r\tau}\mathbb{E}\left[\left(S_t\exp\left\{\left(r-\frac{1}{2}\sigma^2\right)\tau-\sigma\sqrt{\tau}Z\right\}-K\right)^+\right] \\
&\begin{array}{rcl}=&\frac{e^{-r\tau}}{\sqrt{2\pi}}\int_{-\infty}^{\infty}\left(S_{t}\exp\left\{\left(r-\frac12\sigma^{2}\right)\tau-\sigma\sqrt{\tau}x\right\}-K\right)^{+}e^{-\frac12x^{2}}dx\end{array}
\end{aligned}$

The integrand of the integral above is only non-zero if $S_t\exp\left\{\left(r-\frac12\sigma^2\right)\tau-\sigma\sqrt{\tau}x\right\}-K>0.$ This is exactly the case whenever for the integration-variable $x$ we have

$x<\frac{\log\left(\frac{S_t}{K}\right)+\left(r-\frac{1}{2}\sigma^2\right)\tau}{\sigma\sqrt{\tau}}:=d_2$

Let $N( x) : = \frac 1{\sqrt {2\pi}}\int _{- \infty}^xe^{- \frac 12u^2}du$ denote the standard normal cumulative distribution function. Then the integral on the right can easily be rewritten as

$\begin{aligned}\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{d_2}e^{-r\tau}Ke^{-\frac{1}{2}x^2}dx&=e^{-r\tau}KN\left(d_2\right)\end{aligned}$

\begin{aligned}
&\text{The integral on the left is trickier and requires a variable substitution. Define }y=x+\sigma\sqrt{\tau}, \\
&\text{then the integral can be rewritten as}
\end{aligned}

$\begin{aligned}
&\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{d_{2}}e^{-r\tau}S_{t}\exp\left\{\left(r-\frac{1}{2}\sigma^{2}\right)\tau-\sigma\sqrt{\tau}x\right\}e^{-\frac{1}{2}x^{2}}dx \\
&=\frac{S_{t}}{\sqrt{2\pi}}\int_{-\infty}^{d_{2}}\exp\left\{-\frac{1}{2}\left(x+\sigma\sqrt{\tau}\right)^{2}\right\}dx \\
&=\frac{S_{t}}{\sqrt{2\pi}}\int_{-\infty}^{d_{2}+\sigma\sqrt{\tau}}\exp\left\{-\frac{1}{2}y^{2}\right\}dy=S_{t}N(d_{1})
\end{aligned}$

$\mathrm{where~we~inherently~defined~}d_{1}:=d_{2}+\sigma\sqrt{\tau}.\mathrm{Our~final~result~is~what~is~known~as~the~Black-Scholes~formula~for~a~call~option:}$

$\begin{aligned}
V(t)& =S_{t}N\left(d_{1}\right)-e^{-r\tau}KN\left(d_{2}\right)  \\
d_{1}& =\frac{\log\left(\frac{S_t}K\right)+\left(r+\frac12\sigma^2\right)\tau}{\sigma\sqrt{\tau}}  \\
d_{2}& =d_1-\sigma\sqrt{\tau} 
\end{aligned}$

We have $\Delta_{t}:=\frac{\partial C_{t}}{\partial S_{t}}$
<br>Here $C_{t}$ = $V_{t}$
<br>$V(t)=S_tN(d_1)-e^{-rT}KN(d_2)$
<br>N(x) denote the standard normal cumulative distribution function.

Regarding the term $N(x)$, x = d1 or d2, although $x$ is related to the stock price $S_t$, the derivative of $N(x)$ with respect to $S_t$ equals zero. That is because $N(x)$ is the cumulative distribution function of the standard normal distribution, and $x$ incorporates $S_t$ indirectly through $d_1$. However, the impact of $S_t$ on $N(x)$ is not linear and does not directly affect the derivative $\frac{\partial C_t}{\partial S_t}$ which is the Delta of the option.  When we take the derivative of the option price $C_t$ with respect to $S_t$, we are calculating how much the value of $C_t$ changes with a small change in $S_t$. 

So we have $\Delta_{t}=\frac{\partial C_{t}}{\partial S_{t}}=\frac{\partial S_{t}N\left(d_{1}\right)}{\partial S_{t}}=N\left(d_{1}\right)$


### 2.Prove: $P_t=e^{-r\tau}KN\left(-d_2\right)-S_tN\left(-d_1\right)$

We have $V(t)=S_tN(d_1)-e^{-rT}KN(d_2)$ and $C_t-P_t=S_t-e^{-rT}K$<br>
For the risk-neutral price $V(t)$ of the option, we denote $C_t$, as follows:<br>
$P_t=(S_tN(d_1)-e^{-rT}KN(d_2))-S_t+e^{-rT}K$<br>
From the properties of the standard normal cumulative distribution function: $N(-d)=1-N(d) $ <br>
This can be simplified to:<br>
$P_t=e^{-rT}KN(-d_2)-S_tN(-d_1)$

### 3.Use the Euler method to perform a hedging simulation